<a href="https://colab.research.google.com/github/kanishka-47/Age-Calculator/blob/main/docs/visionRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### vision RAG using [VARAG](https://github.com/adithya-s-k/VARAG)


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adithya-s-k/VARAG/blob/main/docs/visionRAG.ipynb)

Requirement to RUN this notebook - Min T4 GPU

In [1]:
!git clone https://github.com/adithya-s-k/VARAG
%cd VARAG
%pwd

Cloning into 'VARAG'...
remote: Enumerating objects: 834, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 834 (delta 22), reused 24 (delta 16), pack-reused 779 (from 1)
Receiving objects: 100% (834/834), 9.92 MiB | 16.82 MiB/s, done.
Resolving deltas: 100% (472/472), done.
/content/VARAG


'/content/VARAG'

In [2]:
!apt-get update && apt-get install -y && apt-get install -y poppler-utils

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,851 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,765 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,137 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,159 kB]
Get:13 http://archive.ub

In [3]:
%pip install -e .

Obtaining file:///content/VARAG
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.9/73.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.5/95.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.2/29.2 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/3

In [4]:
from sentence_transformers import SentenceTransformer
from varag.rag import VisionRAG
from varag.vlms import OpenAI
import lancedb
import os
from dotenv import load_dotenv

os.environ["OPENAI_API_KEY"] = "api-key"

load_dotenv()

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [ ]:
shared_db = lancedb.connect("~/shared_rag_db")

# Initialize VisionRAG and VLM
embedding_model = SentenceTransformer("jinaai/jina-clip-v1", trust_remote_code=True)

vision_rag = VisionRAG(
    image_embedding_model=embedding_model,
    db=shared_db,
    table_name="visionDemo",
)

vlm = OpenAI()

In [ ]:
vision_rag.index(
        "./examples/data",
        overwrite=False,
        recursive=False,
        verbose=True
    )

In [ ]:
query = "What is Colpali"
num_results = 5

results = vision_rag.search(query, k=5)

images = [result["image"] for result in results]

# Display the images
for i, img in enumerate(images, 1):
    print(f"Image {i}:")
    display(img)

In [ ]:
from IPython.display import display, Markdown


response = vlm.query(query, images, max_tokens=1000)


display(Markdown(response))

### Run Gradio Demo

In [ ]:
%cd examples
!python visionDemo.py --share